In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [2]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Data Preprocessing

In [3]:
# Print the column data types
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# Print out the Application_Type value counts
app_counts = charity_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Print out the AFFILIATION value counts
affil_counts = charity_df.AFFILIATION.value_counts()
affil_counts

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [6]:
# Print out the CLASSIFICATION value counts
class_counts = charity_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1732        1
C1570        1
C8210        1
C4120        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Print out the USE_CASE value counts
use_counts = charity_df.USE_CASE.value_counts()
use_counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [8]:
# Print out the ORGANIZATION value counts
org_counts = charity_df.ORGANIZATION.value_counts()
org_counts

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [9]:
# Print out the INCOME_AMT value counts
inc_counts = charity_df.INCOME_AMT.value_counts()
inc_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

Because INCOME_AMT has bucketed ordinal data we will not further combine values and will one hot encode all listed values.

In [10]:
# Define our binning function

def binning(col, val_counts, min_val):
    # Determine which values to replace
    replace_vals = list(val_counts[val_counts < min_val].index)

    # Replace in DataFrame
    for val in replace_vals:
        col = col.replace(val,"Other")
    return col


# Run the binning function on each categorical column for predetermined minimum values
charity_df["APPLICATION_TYPE"] = binning(charity_df["APPLICATION_TYPE"], app_counts, 1000)
charity_df["AFFILIATION"] = binning(charity_df["AFFILIATION"], affil_counts, 10000)
charity_df["CLASSIFICATION"] = binning(charity_df["CLASSIFICATION"], class_counts, 1800)
charity_df["USE_CASE"] = binning(charity_df["USE_CASE"], use_counts, 1000)
charity_df["ORGANIZATION"] = binning(charity_df["ORGANIZATION"], org_counts, 1000)

In [11]:
# Convert the SPECIAL_CONSIDERATIONS column to a dummy
def dummy(col):
    if col == 'N':
        return 0
    else:
        return 1

charity_df["SPECIAL_CONSIDERATIONS"] = charity_df["SPECIAL_CONSIDERATIONS"].apply(dummy)

In [12]:
# Set the DataFrame index to EIN
#charity_df = charity_df.set_index(["EIN"])
#charity_df

In [13]:
char_enc_df = charity_df.drop(columns = ["EIN", "NAME"])
char_cat = char_enc_df.dtypes[char_enc_df.dtypes == "object"].index.tolist()
char_enc_df[char_cat].nunique()

APPLICATION_TYPE    6
AFFILIATION         3
CLASSIFICATION      6
USE_CASE            3
ORGANIZATION        3
INCOME_AMT          9
dtype: int64

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(char_enc_df[char_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(char_cat)
encode_df

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
char_enc_df = char_enc_df.merge(encode_df,left_index=True, right_index=True).drop(char_cat, 1)
char_enc_df

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,1,0,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,1,0,5000,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,1,0,5000,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Split our preprocessed data into our features and target arrays
y = char_enc_df["IS_SUCCESSFUL"].values
X = char_enc_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Creating and Training the Model

In [38]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  48
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 48)                1632      
_________________________________________________________________
dense_24 (Dense)             (None, 20)                980       
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 168       
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 9         
Total params: 2,789
Trainable params: 2,789
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.7351 - accuracy: 0.5399
Loss: 0.7350586034883215, Accuracy: 0.539941668510437


In [41]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)

Train on 25724 samples
Epoch 1/500
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5403 - accuracy: 0.7401
Epoch 2/500
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5404 - accuracy: 0.7397
Epoch 3/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5404 - accuracy: 0.7396
Epoch 4/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5404 - accuracy: 0.7400
Epoch 5/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5401 - accuracy: 0.7397
Epoch 6/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5402 - accuracy: 0.7397
Epoch 7/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5401 - accuracy: 0.7405
Epoch 8/500
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5404 - accuracy: 0.7398
Epoch 9/500
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5398 - accuracy: 0.7400
Epoch

25724/25724 [==============================] - 1s 35us/sample - loss: 0.5387 - accuracy: 0.7402
Epoch 76/500
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5383 - accuracy: 0.7409
Epoch 77/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5389 - accuracy: 0.7398
Epoch 78/500
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5387 - accuracy: 0.7406
Epoch 79/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5383 - accuracy: 0.7406
Epoch 80/500
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5384 - accuracy: 0.7407
Epoch 81/500
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5387 - accuracy: 0.7407
Epoch 82/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5384 - accuracy: 0.7409
Epoch 83/500
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5384 - accuracy: 0.7413
Epoch 84/500
25724/25724 [======

25724/25724 [==============================] - 1s 34us/sample - loss: 0.5376 - accuracy: 0.7412
Epoch 150/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5374 - accuracy: 0.7409
Epoch 151/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5376 - accuracy: 0.7413
Epoch 152/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5376 - accuracy: 0.7406
Epoch 153/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5374 - accuracy: 0.7415
Epoch 154/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5375 - accuracy: 0.7416
Epoch 155/500
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5375 - accuracy: 0.7409
Epoch 156/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5373 - accuracy: 0.7412
Epoch 157/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5373 - accuracy: 0.7416
Epoch 158/500
25724/2572

25724/25724 [==============================] - 1s 57us/sample - loss: 0.5368 - accuracy: 0.7416
Epoch 224/500
25724/25724 [==============================] - 1s 39us/sample - loss: 0.5368 - accuracy: 0.7417
Epoch 225/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5369 - accuracy: 0.7409
Epoch 226/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5365 - accuracy: 0.7416
Epoch 227/500
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5366 - accuracy: 0.7417
Epoch 228/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5365 - accuracy: 0.7413
Epoch 229/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5368 - accuracy: 0.7410
Epoch 230/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5367 - accuracy: 0.7413
Epoch 231/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5364 - accuracy: 0.7414
Epoch 232/500
25724/2572

25724/25724 [==============================] - 1s 34us/sample - loss: 0.5359 - accuracy: 0.7414
Epoch 298/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5360 - accuracy: 0.7419
Epoch 299/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5360 - accuracy: 0.7418
Epoch 300/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5364 - accuracy: 0.7418
Epoch 301/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5360 - accuracy: 0.7420
Epoch 302/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5362 - accuracy: 0.7416
Epoch 303/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5360 - accuracy: 0.7413
Epoch 304/500
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5359 - accuracy: 0.7423
Epoch 305/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5359 - accuracy: 0.7421
Epoch 306/500
25724/2572

25724/25724 [==============================] - 1s 44us/sample - loss: 0.5354 - accuracy: 0.7421
Epoch 372/500
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5356 - accuracy: 0.7425
Epoch 373/500
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5351 - accuracy: 0.7420
Epoch 374/500
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5356 - accuracy: 0.7423
Epoch 375/500
25724/25724 [==============================] - 1s 38us/sample - loss: 0.5355 - accuracy: 0.7423
Epoch 376/500
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5352 - accuracy: 0.7426
Epoch 377/500
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5350 - accuracy: 0.7421
Epoch 378/500
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5358 - accuracy: 0.7421
Epoch 379/500
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5357 - accuracy: 0.7424
Epoch 380/500
25724/2572

25724/25724 [==============================] - 1s 36us/sample - loss: 0.5345 - accuracy: 0.7425
Epoch 446/500
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5348 - accuracy: 0.7428
Epoch 447/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5349 - accuracy: 0.7427
Epoch 448/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5349 - accuracy: 0.7425
Epoch 449/500
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5352 - accuracy: 0.7422
Epoch 450/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5348 - accuracy: 0.7422
Epoch 451/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5348 - accuracy: 0.7427
Epoch 452/500
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5345 - accuracy: 0.7424
Epoch 453/500
25724/25724 [==============================] - 1s 35us/sample - loss: 0.5347 - accuracy: 0.7429
Epoch 454/500
25724/2572

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5703 - accuracy: 0.7242
Loss: 0.5703374858917369, Accuracy: 0.7241982221603394
